# Step 4: Get Walking Distance to the Nearest MRT

## Import Libraries

In [1]:
import pandas as pd
import requests
import json as json
import time
import csv

from myFunctions import *
keys = getKeys('GoogleAPIKey.txt')

## User Inputs
I like to date my output files so that I don't accidentally overwrite files of my previous runs. For more automated systems, it should be quite simple to append today's date to the filename. A function will be needed to append a version number to the date if there is already a file with the same name in the directory. But for developmental purposes, it is better to have the user manually change this so that the user knows what he/she is doing. 

The data file is the output of Step2+3 containing the nearest MRT station to each unique HDB address. 

We need to find the walking distance from 8260 unique HDB addresses to their corresponding nearest MRT. 
So this code will need to be run several times to not exceed the daily API usage limit.  The split can be done by changing the 'start' and 'end' variables which are used to slice the pandas data frame. Note that contrary to usual python slices, both the start and the end are included.

In [2]:
dataFile = 'nearest_MRT_20180311.csv'
outputFile = 'walking_distances_20180318.csv'
outputFailureFile = 'walking_distance_failed_20180318.csv'

# Set data range to query for this run. Total data point is 8260. 
start = 0
end = 2100

## Read Origin and Destination Coordinates
The data frame that is read from the data files are preserved to:
- contain all required columns. 
- be indexed by HDB address/Station Code.  

This is so that details of failed requests can be accessed from this data frame and saved to a failure file.  The failure file can thus be re-run with this same code later. 

In [3]:
raw = pd.read_csv(dataFile)
raw = raw.loc[start:end, :]
index = raw['HDB Address'].tolist()

hdb_latitude = raw['HDB Latitude'].tolist()
hdb_longtitude = raw['HDB Longtitude'].tolist()
hdb_latlong = []
for i in range(len(index)):
    hdb_latlong.append(str(hdb_latitude[i]) + ',' + str(hdb_longtitude[i]))

hdb_coords = dict(zip(index, hdb_latlong))

mrt_code = raw['Nearest Station Code'].tolist()
mrt_latitude = raw['Nearest Station Latitude'].tolist()
mrt_longtitude = raw['Nearest Station Longtitude'].tolist()
mrt_latlong = []
for i in range(len(index)):
    mrt_latlong.append([mrt_code[i], str(mrt_latitude[i]) + ',' + str(mrt_longtitude[i])])

mrt_coords = dict(zip(index, mrt_latlong))

# Preserve data for failure retry later
raw = raw.loc[:, ['HDB Address', 'HDB Latitude', 'HDB Longtitude', 'Nearest Station Code', 'Nearest Station Latitude', 'Nearest Station Longtitude']]
raw.set_index('HDB Address', inplace = True)

## API Query
The outermost ***with*** loop ensures that retrieved data are all written to a file if the code is interrupted.   

The ***success*** and ***failure*** dictionaries store the walking distance and duration/error for each HDB address. This is to ease troubleshooting. In the event that these dictionaries take up too much memory space, the ***success*** dictionary can be removed because the data are already written to file in the ***with*** loop. However, the ***failure*** dictinary needs to be kept as the data are converted to a data frame and file at the end of the code. 

In [4]:
success = {}
failure = {}
url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'

In [5]:
with open(outputFile, 'w', newline = '', encoding = 'utf-8') as output:
    writer = csv.writer(output, delimiter=',')
    writer.writerow(['HDB Address', 'MRT Station Code', 'Walking Distance in Meters ', 'Walking Duration in Minutes'])

    for h in mrt_coords:
        m = mrt_coords[h]
        params = {'key': keys['GoogleMapsDistanceMatrix'],
                  'origins': hdb_coords[h],
                  'destinations': m[1],
                  'mode': 'walking'}
        paramString = getParamString(params)
        reqStatus, returnedStatus, data, tries = tryGET(3, url+paramString)
    
        if reqStatus == 200 and returnedStatus =='OK' and data['rows'][0]['elements'][0]['status'] == 'OK':
            walking_dist_meters = data['rows'][0]['elements'][0]['distance']['value']
            walking_duration_mins = data['rows'][0]['elements'][0]['duration']['value']/60
            success[h] = [walking_dist_meters, walking_duration_mins]
            writer.writerow([h,m[0], walking_dist_meters, walking_duration_mins])
            print('Walking distance retrieved for %s. ' %h)
        else:
            failure[h] = [raw.loc[h,'HDB Latitude'], raw.loc[h,'HDB Longtitude'], raw.loc[h,'Nearest Station Code'], raw.loc[h,'Nearest Station Latitude'], raw.loc[h,'Nearest Station Longtitude'], reqStatus, returnedStatus]
            print('Walking distance retrieve error for %s. %s. %s' %(h, str(reqStatus),str(returnedStatus)))

print('Number of success: ', len(success))
print('Number of failures: ', len(failure))

Try: 1. 
Walking distance retrieved for Blk+83+COMMONWEALTH+CL+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+97+COMMONWEALTH+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+95+COMMONWEALTH+DR+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+98+COMMONWEALTH+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+93+COMMONWEALTH+DR+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+85+COMMONWEALTH+CL+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+101+COMMONWEALTH+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+90+COMMONWEALTH+DR+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+7+COMMONWEALTH+AVE+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+410+COMMONWEALTH+AVE+WEST+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+110+COMMONWEALTH+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+100+COMMONWEALTH+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+88+COMMONWEALTH+CL+Singapore. 
Try: 

Walking distance retrieved for Blk+542+BEDOK+NTH+ST+3+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+76+BEDOK+NTH+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+709+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+703+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+27+NEW+UPP+CHANGI+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+42+CHAI+CHEE+ST+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+426+BEDOK+NTH+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+530+BEDOK+NTH+ST+3+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+420+BEDOK+NTH+ST+1+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+105+BEDOK+NTH+AVE+4+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+519+BEDOK+NTH+AVE+1+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+548+BEDOK+NTH+AVE+1+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+537+BEDOK+NTH+ST+3+Singapore. 
Try: 1. 
Walking distance ret

Walking distance retrieved for Blk+146+BT+BATOK+WEST+AVE+6+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+533+BT+BATOK+ST+51+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+157+BT+BATOK+ST+11+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+540+BT+BATOK+ST+52+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+239+BT+BATOK+EAST+AVE+5+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+249+BT+BATOK+EAST+AVE+5+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+316+BT+BATOK+ST+32+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+250+BT+BATOK+EAST+AVE+5+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+164+BT+BATOK+ST+11+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+167+BT+BATOK+WEST+AVE+8+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+312+BT+BATOK+ST+32+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+208+BT+BATOK+ST+21+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+534+BT+BATOK+ST+51+Singapore. 
Try: 1. 


Walking distance retrieved for Blk+448+BT+PANJANG+RING+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+507+JELAPANG+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+478+SEGAR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+625+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+629+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+623+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+470+SEGAR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+610+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+611+BT+PANJANG+RING+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+627+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+626+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+608+SENJA+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+124+PENDING+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+653+SENJA+LINK+Singapore. 
Try: 1. 
Walking

Walking distance retrieved for Blk+43+CIRCUIT+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+58+CIRCUIT+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+109+ALJUNIED+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+61+CIRCUIT+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+20+BALAM+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+93+PAYA+LEBAR+WAY+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+25+EUNOS+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+23+BALAM+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+21+EUNOS+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+9+HAIG+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+16+EUNOS+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+102+ALJUNIED+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+31+BALAM+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+126+GEYLANG+EAST+AVE+1+Singapore. 
Try: 1. 
Walking d

...

Try: 1. 
Walking distance retrieved for Blk+631+PASIR+RIS+DR+3+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+416+PASIR+RIS+DR+6+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+149+PASIR+RIS+ST+13+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+769+PASIR+RIS+ST+71+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+233+PASIR+RIS+DR+4+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+513+PASIR+RIS+ST+52+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+773+PASIR+RIS+ST+71+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+479+PASIR+RIS+DR+4+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+477+PASIR+RIS+DR+6+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+119+PASIR+RIS+ST+11+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+194+PASIR+RIS+ST+12+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+612+ELIAS+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+113+PASIR+RIS+ST+11+Singapore. 
Try: 1. 
Walking distanc

Walking distance retrieved for Blk+339A+SEMBAWANG+CL+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+315+SEMBAWANG+VISTA+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+467B+ADMIRALTY+DR+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+295A+COMPASSVALE+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+124B+RIVERVALE+DR+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+295B+COMPASSVALE+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+309C+ANCHORVALE+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+308B+ANCHORVALE+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+159A+RIVERVALE+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+158B+RIVERVALE+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+188A+RIVERVALE+DR+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+184C+RIVERVALE+CRES+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+126+RIVERVALE+ST+Singapore. 
Try: 1. 
Walking distance retr

Try: 1. 
Walking distance retrieved for Blk+864+TAMPINES+ST+83+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+858A+TAMPINES+AVE+5+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+862A+TAMPINES+ST+83+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+144+TAMPINES+ST+12+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+896+TAMPINES+ST+81+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+868+TAMPINES+ST+83+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+711+TAMPINES+ST+71+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+247+TAMPINES+ST+21+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+874+TAMPINES+ST+84+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+391+TAMPINES+AVE+7+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+279+TAMPINES+ST+22+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+157+SIMEI+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+501+TAMPINES+CTRL+1+Singapore. 
Try: 1. 
Walking distance re

Walking distance retrieved for Blk+780+YISHUN+RING+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+261+YISHUN+ST+22+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+711+YISHUN+AVE+5+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+789+YISHUN+AVE+2+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+220+YISHUN+ST+21+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+140+YISHUN+RING+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+283+YISHUN+AVE+6+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+210+YISHUN+ST+21+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+216+YISHUN+ST+21+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+214+YISHUN+ST+21+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+250+YISHUN+AVE+9+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+117+YISHUN+RING+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+134+YISHUN+ST+11+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+329+YISHUN

Try: 1. 
Walking distance retrieved for Blk+117+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+123+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+54+CHAI+CHEE+ST+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+149+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+132+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+707+BEDOK+NTH+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+513+BEDOK+NTH+AVE+2+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+201+BEDOK+NTH+ST+1+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+623+BEDOK+RESERVOIR+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+78+BEDOK+NTH+RD+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+95+BEDOK+NTH+AVE+4+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+169+BEDOK+STH+AVE+3+Singapore. 
Try: 1. 
Walking distance retrieved for Blk+408+BEDOK+NTH+AVE+2+Singapore. 
Try: 1. 
W

## Save Failure Data

In [6]:
if len(failure)!=0:
    failure = pd.DataFrame.from_dict(failure, orient = 'Index')
    failure.reset_index(level=0, inplace=True)
    failure.columns = ['HDB Address', 'HDB Latitude', 'HDB Longtitude', 'Nearest Station Code', 'Nearest Station Latitude','Nearest Station Longtitude','reqStatus', 'returnedStatus']
    failure.to_csv(outputFailureFile, index = False)